# A Mixed Integer Programming Model for Timed Deliveries in Multirobot Systems

#### Nitin Kamra and Nora Ayanian

## Problem Description

Consider a 2-D grid with a control center at the origin O(0, 0), $m$ delivery robots and $n$ task robots whose locations
are described relative to the control center.

New incoming delivery requests from task robots are collected and added to a list of requests. Requests are processed to keep track of “resource cycles” for robots that periodically need resources with a fixed lifespan e.g., batteries.

We seek to generate a schedule for delivery robots to serve the maximum number of requests possible, while both minimizing the offset from expected delivery times and taking task robot priorities into account.

## Model Formulation

### Sets

- $K$: Set of delivery robots. $|K| = M$.
- $T$ : Set of task robots. $|T| = N$.
- $V = K \cup T$. $|V|=M+N$.

All elements of $V$ are added to the graph $G$ as nodes.

Two nodes, located at the control center, are also added to $G$. These nodes are called $Start(\alpha)$ and $End(\omega)$. Delivery robots are assumed to begin at the start node $Start(\alpha)$. At the end of the time window, all delivery robots must return to the end node $End(\omega)$.

Hence the full set of nodes is $M+N+2$, denoted by $\mathcal{N}$.


Now, within these nodes:
- All task robots $T$ have directed edges to each other and to $End$ node (with weight $t_{ij} = d_{ij}/v$) .
- $Start$ node has edges leading to all delivery robots, which are considered pre-traversed ($t_{\alpha k}=0$).
- Each delivery robot has an edge to each task robot and to $End$ node.

### Parameters

- $t_{ij}$ : Time to travel from node $i$ to node $j$.
- $\tau_i$ : Time when $i^{th}$ task robot requires delivery.
- $T_s$ : Minimum stay time for a delivery robot to make a delivery; they may stay longer if needed. Can be increased to allow small movements of task robots.
- $C^k$ : Maximum carrying capacity of the $k^th$ delivery robot.
- $c^k$ : No. of resources already delivered by $k^{th}$ delivery robot during this time window.
- $B^k$ : Fraction of total battery power remaining on the $k^{th}$ delivery robot.
- $B^k_r$ : Average battery usage per unit distance travelled for the $k^{th}$ delivery robot, as fraction of total battery power.
- $p_i \in [0,1]$ : Priority of task robot $i$.
- $T_{A,i}$ : Advance delivery time for $i^{th}$ task robot.
- $Z$ : A very large positive constant.
- $T_{start}$ : Time at which dispatch begins.
- $T_W$ : Length of current time window.
- $t_{bound}$ : Final time by which all delivery robots must reach the $End$ node. We set $t_{bound} = 3T_W/2$
- $\lambda$ : Control parameter governing the trade-off in the objective function.

### Variables

A single scheduling window is considered as one optimization problem on the graph $G$.

The variables are:

- $x_{ij}^k \in {0,1}$ : Binary. '$1$' only if $k^{th}$ delivery robot travels from node $i$ to $j$.
- $a_i \in \mathbb{R}^+$ : Arrival time at node $i \in (V - K)$
- $d_i \in \mathbb{R}^+$ : Departure time from node $i \in V$ 

### Objective function

$$\min\limits_{x, a, d} \{f_{time}(d) + \lambda f_{travel}(x,a,d)\}        \tag{1}$$

where

$$f_{time}(d) = \sum\limits_{i \in (V-K)}p_i(d_i - ( \tau_i - T_{A,i}))^2$$

and

$$f_{travel}(x,a,d) = \sum\limits_{k \in K}\left(\sum_{\substack{(i,j) \in E \\ j \neq \omega}} x_{ij}^k(a_j -di) + \sum_{\substack{(i,j) \in E \\ j = \omega}} x_{ij}^kt_{ij}\right)$$

### Constraints

#### Path Continuity Constraints:

- Each node $i$ should be visited by either only exactly 1 delivery robot or none.

$$\sum_{k \in K} \sum_{j \in V \cup \{w\}} x_{ij}^k \leq 1 \quad \forall i \in V \tag{2}$$

Allowing no visits provides flexibility if all nodes couldn't be served in the given time window.

- Impose pre-traversal from $Start$ node for each delivery robot.

$$x_{\alpha k} = 1 \quad \forall k \in K \tag {3}$$

- Require delivery robots to arrive at $End$ nodes.

$$\sum_{i \in V} x^k_{i\omega} = 1 \quad \forall k \in K \tag {4}$$

- If a delivery robot enters a node $h$, it should leave node $h$.

$$\sum_{i \in \{\alpha\} \cup V} x_{ih}^k = \sum_{j \in \{V - K\} \cup \omega} x_{hj}^k, \quad \forall h \in V, \forall k \in K \tag{4} $$

#### TIme Flow Constraints:

- Arrival and departure times by the delivery robot at a task robot node should differ by at least $T_s$ to swap battery.

$$a_i - d_i + T_s \leq 0 \quad \forall i \in (V-K) \tag{6}$$

- Departure from node $i$ and arrival at node $j$ must differ by at least the time $t_{ij}$ it takes to traverse the edge .

$$d_i - a_j + t_{ij} \leq Z \left(1 - \sum_{k \in K}x_{ij}^k  \right) \quad \forall (i,j) \in E, j\neq \omega \tag{7}$$

Large positive constant $Z$ ensures that if $(i, j)$ is not being traversed, the constraint is trivially satisfied.

- Bound arrival and departure times. $t_{bound} > T_W$ gives delivery robots sufficient time to return bcak to control center.

$$T_{start} \leq a_i \leq T_{start} + t_{bound} \quad \forall i \in (V-K) \tag{8}$$

$$T_{start} \leq d_i \leq T_{start} + t_{bound} \quad \forall i \in V \tag{9}$$

- $(10)$ coupled with $(9)$, ensure that departure time for node $i$ that is not visited by any delivery robot is $T_{start} + t_{bound}$. This also penalizes heavily for not serving robot $i$.

$$T_{start} + t_{bound}\left(1-\sum_{k \in K} \sum_{j \in V \cup \omega} x_{ij}^k \right) \leq d_i \quad \forall i \in V \tag{10}$$

- A delivery robot returning directly to the control center should start immediately at the begining of dispatch.

$$d_k - Z(1-x^k_{k\omega}) \leq T_{start} \quad \forall k \in K \tag{11}$$

without this constraint, delivery robots directly returning to the control center will have their departure time set as $T_{start} + t_{bound}$, resulting in stagnation when the window shifts.

#### Capacity Constraints:

- Prevent a robot from delivering more batteries than it is carrying.
$$\sum_{(i,j) \in E}x_{ij}^k - 1 \leq C^k - c^k \quad \forall k \in K \tag{12}$$

- Delivery robots should return back to control center before depletig their own batteries.
$$\sum_{(i,j) \in E} x^k_{ij}B_r^k(t_{ij}v) \leq B^k \quad \forall k \in K \tag{13}$$

## Python Implementation


Import `gurobipy` and other modules:

In [1]:
from gurobipy import *
import random as rnd
import math
import pprint
import numpy as np
import pixiedust

Pixiedust database opened successfully


### Data definition

Define sets $K$, $T$ and $V$:

In [2]:
K = ["K1", "K2"]
T = ["T1", "T2", "T3"]
startNode = ["Start"]
endNode = ["End"]
V = K + T
N = V + startNode + endNode
print(N)

['K1', 'K2', 'T1', 'T2', 'T3', 'Start', 'End']


Lets define locations of these sets as `dicts`

In [3]:
N = {nodeID: (100*rnd.random(), 100*rnd.random()) for nodeID in N }
pprint.pprint(N)

{'End': (77.03926045108022, 65.26890998855482),
 'K1': (29.26085208764967, 37.55902528351535),
 'K2': (33.411929810262166, 1.029048367048857),
 'Start': (83.26519676445517, 3.40765633068707),
 'T1': (68.53991252674112, 98.5208506680894),
 'T2': (59.27422788140993, 76.82491381611615),
 'T3': (80.92025915671542, 18.6763458380528)}


Lets define the arcs from '$Start$' node to all delivery robots:

In [4]:
arcsStartK = [('Start', i) for i in K]
# print(arcsStartK)

Add weight to arcsStartK

In [5]:
iter_arcsStartK = iter(arcsStartK)
arcsStartK = tupledict([(x, 0) for x in iter_arcsStartK])
print(arcsStartK)

{('Start', 'K1'): 0, ('Start', 'K2'): 0}


Lets add edges from delivery robots to task robots and to end node

In [6]:
arcsKTEnd = [(i,j) for i in K for j in T + endNode]
print (arcsKTEnd)

[('K1', 'T1'), ('K1', 'T2'), ('K1', 'T3'), ('K1', 'End'), ('K2', 'T1'), ('K2', 'T2'), ('K2', 'T3'), ('K2', 'End')]


Add weights to arcsKTEnd

In [7]:
iter_arcsKTEnd = iter(arcsKTEnd)
arcsKTEnd = {t: np.linalg.norm(np.array(N.get(t[0]))-np.array(N.get(t[1]))) for t in iter_arcsKTEnd} 
pprint.pprint(arcsKTEnd)

{('K1', 'End'): 55.232363846836314,
 ('K1', 'T1'): 72.52026436243248,
 ('K1', 'T2'): 49.42279563917955,
 ('K1', 'T3'): 55.00227196914264,
 ('K2', 'End'): 77.65374298765964,
 ('K2', 'T1'): 103.627345259996,
 ('K2', 'T2'): 80.08665107678061,
 ('K2', 'T3'): 50.680059839360624}


Lets also add edges from task robots to task robots and to end nodes

In [8]:
arcsTTEnd = [(i,j) for i in T for j in T + endNode if i!=j]
iter_arcsTTEnd = iter(arcsTTEnd)
arcsTTEnd = {t: np.linalg.norm(np.array(N.get(t[0]))-np.array(N.get(t[1]))) for t in iter_arcsTTEnd}
pprint.pprint(arcsTTEnd)

{('T1', 'End'): 34.320991741123294,
 ('T1', 'T2'): 23.591663524040367,
 ('T1', 'T3'): 80.79862581895841,
 ('T2', 'End'): 21.192866881667012,
 ('T2', 'T1'): 23.591663524040367,
 ('T2', 'T3'): 62.04680997336574,
 ('T3', 'End'): 46.75392160099288,
 ('T3', 'T1'): 80.79862581895841,
 ('T3', 'T2'): 62.04680997336574}


Combine the three dictionaries to get the final set of arcs

In [9]:
arcs = {**arcsStartK, **arcsTTEnd, **arcsKTEnd}
arcs = tupledict(arcs)
pprint.pprint(arcs)

{('K1', 'End'): 55.232363846836314,
 ('K1', 'T1'): 72.52026436243248,
 ('K1', 'T2'): 49.42279563917955,
 ('K1', 'T3'): 55.00227196914264,
 ('K2', 'End'): 77.65374298765964,
 ('K2', 'T1'): 103.627345259996,
 ('K2', 'T2'): 80.08665107678061,
 ('K2', 'T3'): 50.680059839360624,
 ('Start', 'K1'): 0,
 ('Start', 'K2'): 0,
 ('T1', 'End'): 34.320991741123294,
 ('T1', 'T2'): 23.591663524040367,
 ('T1', 'T3'): 80.79862581895841,
 ('T2', 'End'): 21.192866881667012,
 ('T2', 'T1'): 23.591663524040367,
 ('T2', 'T3'): 62.04680997336574,
 ('T3', 'End'): 46.75392160099288,
 ('T3', 'T1'): 80.79862581895841,
 ('T3', 'T2'): 62.04680997336574}


Lets create sets of arcs for each charger robot

In [10]:
#kArcs = [(k,arc) for k in K for arc in arcs if arc[0]!='Start' or arc[0] in ]
kArcs=[]
for k in K:
    for arc in arcs:
        if arc[0]!='Start':
            kMinusK= list(set(K) - set([k]))
            if arc[0] not in kMinusK:
                kArcs.append((k,arc))
        


pprint.pprint(kArcs)

[('K1', ('T1', 'T2')),
 ('K1', ('T1', 'T3')),
 ('K1', ('T1', 'End')),
 ('K1', ('T2', 'T1')),
 ('K1', ('T2', 'T3')),
 ('K1', ('T2', 'End')),
 ('K1', ('T3', 'T1')),
 ('K1', ('T3', 'T2')),
 ('K1', ('T3', 'End')),
 ('K1', ('K1', 'T1')),
 ('K1', ('K1', 'T2')),
 ('K1', ('K1', 'T3')),
 ('K1', ('K1', 'End')),
 ('K2', ('T1', 'T2')),
 ('K2', ('T1', 'T3')),
 ('K2', ('T1', 'End')),
 ('K2', ('T2', 'T1')),
 ('K2', ('T2', 'T3')),
 ('K2', ('T2', 'End')),
 ('K2', ('T3', 'T1')),
 ('K2', ('T3', 'T2')),
 ('K2', ('T3', 'End')),
 ('K2', ('K2', 'T1')),
 ('K2', ('K2', 'T2')),
 ('K2', ('K2', 'T3')),
 ('K2', ('K2', 'End'))]


Lets add two more arcs from $Start$ to delivery robots. (Because the formulation requires I guess)

In [11]:
finalArcs = kArcs + [arc for arc in arcsStartK]
pprint.pprint(finalArcs)

[('K1', ('T1', 'T2')),
 ('K1', ('T1', 'T3')),
 ('K1', ('T1', 'End')),
 ('K1', ('T2', 'T1')),
 ('K1', ('T2', 'T3')),
 ('K1', ('T2', 'End')),
 ('K1', ('T3', 'T1')),
 ('K1', ('T3', 'T2')),
 ('K1', ('T3', 'End')),
 ('K1', ('K1', 'T1')),
 ('K1', ('K1', 'T2')),
 ('K1', ('K1', 'T3')),
 ('K1', ('K1', 'End')),
 ('K2', ('T1', 'T2')),
 ('K2', ('T1', 'T3')),
 ('K2', ('T1', 'End')),
 ('K2', ('T2', 'T1')),
 ('K2', ('T2', 'T3')),
 ('K2', ('T2', 'End')),
 ('K2', ('T3', 'T1')),
 ('K2', ('T3', 'T2')),
 ('K2', ('T3', 'End')),
 ('K2', ('K2', 'T1')),
 ('K2', ('K2', 'T2')),
 ('K2', ('K2', 'T3')),
 ('K2', ('K2', 'End')),
 ('Start', 'K1'),
 ('Start', 'K2')]


#### Parameters
- $t_{ij}$ : Time to travel from node $i$ to node $j$.
- $\tau_i$ : Time when $i^{th}$ task robot requires delivery.
- $T_s$ : Minimum stay time for a delivery robot to make a delivery; they may stay longer if needed. Can be increased to allow small movements of task robots.
- $C^k$ : Maximum carrying capacity of the $k^{th}$ delivery robot.
- $c^k$ : No. of resources already delivered by $k^{th}$ delivery robot during this time window.
- $B^k$ : Fraction of total battery power remaining on the $k^{th}$ delivery robot.
- $B^k_r$ : Average battery usage per unit distance travelled for the $k^{th}$ delivery robot, as fraction of total battery power.
- $p_i \in [0,1]$ : Priority of task robot $i$.
- $T_{A,i}$ : Advance delivery time for $i^{th}$ task robot.
- $Z$ : A very large positive constant.
- $T_{start}$ : Time at which dispatch begins.
- $T_W$ : Length of current time window.
- $t_{bound}$ : Final time by which all delivery robots must reach the $End$ node. We set $t_{bound} = 3T_W/2$
- $\lambda$ : Control parameter governing the trade-off in the objective function.

In [12]:
t_ij = arcs                                            # Assuming v = 1
T_W = 100000                                           # Lets assume a time window of 10000s
tau_i = {nodeID: T_W*rnd.random() for nodeID in T}     # Assign random delivery times required for task robot i
T_s = 10                                               # Assume 10s for Battery swap time
C_k = 20                                               # Assume each delivery robot can carry 20 batteries
cc_k = 5                                               # Assume 5 batteries have already been delivered by this robot.
B_k = 10000                                            # Assume delivery robot can stay up for 10000s
B_k_r = 1                                              # Assume 1 unit per travel distance uni
p_i = {nodeID:rnd.random() for nodeID in T}                        # Assign random priorities to each task robot
T_start = 1
T_A_i = {nodeID: max(T_start+200 + 
             T_W*rnd.random(), T_W) for nodeID in T}   # Advance delivery time assigned to each task robot, 
                                                       # min T_start + 200 max T_W
Z = 1e10                                               # A very large constant
t_bound = 3*T_W/2                                      # Final time by which all delivery robots must reach the End node
param_lambda = 0.4                                     # Randomly chosen tradeoff parameter for now
pprint.pprint(p_i)

{'T1': 0.8493957009962281, 'T2': 0.397931571743358, 'T3': 0.2775036254897264}


### Model Generation

Create empty named model object:

In [13]:
#model = Model('FuelConstrainedRobots')

#### Decision Variables
Following are the decision variables (discussed above)
- $x_{ij}^k \in {0,1}$ : Binary. '$1$' only if $k^{th}$ delivery robot travels from node $i$ to $j$.
- $a_i \in \mathbb{R}^+$ : Arrival time at node $i \in (V - K)$
- $d_i \in \mathbb{R}^+$ : Departure time from node $i \in V$ 

In [14]:
model = Model('FuelConstrainedRobots')
x = model.addVars(finalArcs, name="x", vtype=GRB.BINARY)
# Generate Arrival Times Array, and then addVars
VminusK = T  + endNode
a = [(i,j) for i in K for j in VminusK]
a = model.addVars(a, name = "a")
# Generate Departure Times Array, and then addVars
d = [(i,j) for i in K for j in T]
# Also add departures from charger's own respective nodes
d = d + [(i,j) for i in K for j in K if i==j]
d = model.addVars(d, name = "d")


# Print variable Names
model.optimize()
print("\n\nVariable Names")
print("--------------")
for v in model.getVars():
    print(v.VarName)

Academic license - for non-commercial use only
Optimize a model with 0 rows, 44 columns and 0 nonzeros
Variable types: 16 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


Variable Names
--------------
x[K1,('T1', 'T2')]
x[K1,('T1', 'T3')]
x[K1,('T1', 'End')]
x[K1,('T2', 'T1')]
x[K1,('T2', 'T3')]
x[K1,('T2', 'End')]
x[K1,('T3', 'T1')]
x[K1,('T3', 'T2')]
x[K1,('T3', 'End')]
x[K1,('K1', 'T1')]
x[K1,('K1', 'T2')]
x[K1,('K1', 'T3')]
x[K1,('K1', 'End')]
x[K2,('T1', 'T2')]
x[K2,('T1', 'T3')]
x[K2,('T1', 'End')]
x[K2,('T2', 'T1')]
x[K2,('T2', 'T3')]
x[K2,('T2

#### Objective Function
As discussed above

$$\min\limits_{x, a, d} \{f_{time}(d) + \lambda f_{travel}(x,a,d)\}        \tag{1}$$

where

$$f_{time}(d) = \sum\limits_{i \in (V-K)}p_i(d_i - ( \tau_i - T_{A,i}))^2$$

and

$$f_{travel}(x,a,d) = \sum\limits_{k \in K}\left(\sum_{\substack{(i,j) \in E \\ j \neq \omega}} x_{ij}^k(a_j -d_i) + \sum_{\substack{(i,j) \in E \\ j = \omega}} x_{ij}^kt_{ij}\right)$$

In [15]:
#%%pixie_debugger
fTime = quicksum(p_i[ID]*(d[k,ID] - (tau_i[ID] - T_A_i[ID])) for ID in T for k in K)
#print(fTime)
#for nodePair in arcs.keys():
#    print(nodePair)
# Lets create two intermediate variables for the second expression
varNotEnd = quicksum( x[k, nodePair]*(a[k, nodePair[1]]-d[k, nodePair[0]]) for (k, nodePair) in kArcs 
                     if nodePair[1]!='End' and nodePair[0] in V and nodePair[1] in V )

varEnd =  quicksum( x[k, nodePair]*t_ij[nodePair] for k, nodePair in kArcs 
                     if nodePair[1]=='End' and nodePair[0] in N and nodePair[1] in N)

fTravel = varNotEnd + varEnd

objFun = fTime + param_lambda*fTravel

model.setObjective(objFun, GRB.MINIMIZE)

# Print variable Names
#model.optimize()
#print("\n\nVariable Names")
#print("--------------")
#for v in model.getVars():
#    print(v.VarName)
#print(varNotEnd)
#print("\n\n\n")
#pprint.pprint(varNotEnd)
#pprint.pprint(varEnd)
#pprint.pprint(fTravel)


#### Path Continuity Constraints:
- Each node $i$ should be visited by either only exactly 1 delivery robot or none.

$$\sum_{k \in K} \sum_{j \in V \cup \{w\}} x_{ij}^k \leq 1 \quad \forall i \in V \tag{2}$$

Allowing no visits provides flexibility if all nodes couldn't be served in the given time window.

(As I understand, this mathematical form actually says that each 'edge' should be visited by at most one delivery robot.)


In [16]:
c2 = model.addConstrs( (quicksum(x[k,nodePair]
                        for k, nodePair in kArcs if z==nodePair ) <= 1 
                              for z in [r[1] for r in kArcs ]), name="c2")
#model.write("x.lp")
# Verified by inspection

- Impose pre-traversal from $Start$ node for each delivery robot.

$$x_{\alpha k} = 1 \quad \forall k \in K \tag {3}$$

In [17]:
#%%pixie_debugger
expr = LinExpr(0.0)
print(expr)
for nodePair in arcsStartK:
    expr.addTerms(1.0, x[nodePair])
    print(expr)
        


#expr = (x[k, nodePair] == 1 for k,nodePair in kArcs
#                       if nodePair[0] == 'Start' and nodePair[1] in K)
#    
#print (expr)
#c3 = model.addConstrs(x[k, nodePair] == 1 for k,nodePair in kArcs
#                       if nodePair[0] == 'Start' and nodePair[1] in K , name="c3")
c3 = model.addConstrs(((x[nodePair] == 1) for nodePair in arcsStartK), name="c3")

#model.write("x.lp")
# Vrified by inspection

<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: x[Start,K1]>
<gurobi.LinExpr: x[Start,K1] + x[Start,K2]>


- Require delivery robots to arrive at $End$ nodes.

$$\sum_{i \in V} x^k_{i\omega} = 1 \quad \forall k \in K \tag {4}$$


In [18]:
c4 = model.addConstrs((quicksum(x[k, nodePair] 
                        for k, nodePair in kArcs if nodePair[1] =='End' and z==k) == 1 for z in K),
                                              name="c4")
#model.write("x.lp")
#Verified by inspection

- If a delivery robot enters a node $h$, it should leave node $h$.

$$\sum_{i \in \{\alpha\} \cup V} x_{ih}^k = \sum_{j \in \{V - K\} \cup \omega} x_{hj}^k, \quad \forall h \in V, \forall k \in K \tag{5} $$

In [19]:


c5 = model.addConstrs((quicksum(x[k,(i,h)] for i in V if i!=h and i not in list(set(K) - set([k]))) == 
                      quicksum(x[k,(h,j)] for j in VminusK if h!=j and h not in list(set(K) - set([k]))) 
                       for h in T for k in K), name="c5")
#model.write("x.lp")
# Seems correct

#### TIme Flow Constraints:

- Arrival and departure times by the delivery robot at a task robot node should differ by at least $T_s$ to swap battery.

$$a_i - d_i + T_s \leq 0 \quad \forall i \in (V-K) \tag{6}$$


In [20]:
c6 = model.addConstrs((d[k, i]-a[k, i]-T_s <= 0 for i in T for k in K), name="c6")
#model.write("x.lp")
#Verified by inspection

- Departure from node $i$ and arrival at node $j$ must differ by at least the time $t_{ij}$ it takes to traverse the edge .

$$d_i - a_j + t_{ij} \leq Z \left(1 - \sum_{k \in K}x_{ij}^k  \right) \quad \forall (i,j) \in E, j\neq \omega \tag{7}$$


In [21]:
c7 = model.addConstrs((d[k, nodePair[0]]-a[k, nodePair[1]]+t_ij[nodePair] 
                       <= Z*(1 - (x[k,nodePair])) for k,nodePair in kArcs 
                                   if nodePair[0] not in K and nodePair[1] not in K and nodePair[1]!='End'), name="c7")
#model.write("x.lp")
# Inspected

Large positive constant $Z$ ensures that if $(i, j)$ is not being traversed, the constraint is trivially satisfied.

- Bound arrival and departure times. $t_{bound} > T_W$ gives delivery robots sufficient time to return bcak to control center.

$$T_{start} \leq a_i \leq T_{start} + t_{bound} \quad \forall i \in (V-K) \tag{8}$$

$$T_{start} \leq d_i \leq T_{start} + t_{bound} \quad \forall i \in V \tag{9}$$


In [197]:
c8 = model.addConstrs(((T_start <= a[k,i] <= T_start + t_bound) for i in VminusK for k in K), name="c8")
c9 = model.addConstrs(((T_start <= d[k,i] <= T_start + t_bound) 
                       for i in V for k in K if i not in list(set(K) - set([k]))), name="c9")
#model.write("x.lp")
# Visually Inspected

- $(10)$ coupled with $(9)$, ensure that departure time for node $i$ that is not visited by any delivery robot is $T_{start} + t_{bound}$. This also penalizes heavily for not serving robot $i$.

$$T_{start} + t_{bound}\left(1-\sum_{k \in K} \sum_{j \in V \cup \omega} x_{ij}^k \right) \leq d_i \quad \forall i \in V \tag{10}$$


In [198]:
TplusEndNode = T+endNode
c10 = model.addConstrs((T_start + t_bound*(1-quicksum(x[k,(i,j)] for j in TplusEndNode if i!=j)) <= d[k,i] 
                                                for i in T for k in K), name="c10")
#model.write("x.lp")
# Seems right

- A delivery robot returning directly to the control center should start immediately at the begining of dispatch.

$$d_k - Z(1-x^k_{k\omega}) \leq T_{start} \quad \forall k \in K \tag{11}$$

In [199]:
c11 = model.addConstrs((d[k,k] - Z*(1-x[k, (k,'End')]) <= T_start for k in K), name="c11")
#model.write("x.lp")
#Verified by inspection

without this constraint, delivery robots directly returning to the control center will have their departure time set as $T_{start} + t_{bound}$, resulting in stagnation when the window shifts.

#### Capacity Constraints:

- Prevent a robot from delivering more batteries than it is carrying.
$$\sum_{(i,j) \in E}x_{ij}^k - 1 \leq C^k - c^k \quad \forall k \in K \tag{12}$$


The above constraint does noe seem to consider the fact that no battery is required for the last edge when travelling to the end node. In the following implementation I am considering that as experiment.

In [200]:
c12 = model.addConstrs((quicksum(x[k, nodePair] for k,nodePair in kArcs if k == z and nodePair[1]!='End') - 1 
                       <= C_k - cc_k for z in K), name="c12")
# model.write("x.lp")
# Verified by inspection

- Delivery robots should return back to control center before depletig their own batteries.
$$\sum_{(i,j) \in E} x^k_{ij}B_r^k(t_{ij}v) \leq B^k \quad \forall k \in K \tag{13}$$

In [201]:
#%%pixie_debugger
c13 = model.addConstrs((quicksum(x[k,nodePair]*B_k_r*t_ij[nodePair]*1 
                            for k, nodePair in kArcs if nodePair[0] not in list(set(K) - set([k])) and z==k ) <= B_k 
                                        for z in K ) , name="c13") # Assuming v=1
#model.write("x.lp")
# Verified by inspection

# Solving the model

In [202]:
model.optimize()

Optimize a model with 82 rows, 44 columns and 226 nonzeros
Model has 36 quadratic objective terms
Variable types: 16 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [5e-01, 4e+01]
  QObjective range [8e-01, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start did not produce a new incumbent solution
MIP start violates constraint c3[Start,K1] by 1.000000000

Found heuristic solution: objective 633113.20628
Presolve removed 39 rows and 4 columns
Presolve time: 0.03s
Presolved: 79 rows, 76 columns, 246 nonzeros
Found heuristic solution: objective 633112.40628
Variable types: 42 continuous, 34 integer (26 binary)
Found heuristic solution: objective 513112.40628

Root relaxation: objective 6.247364e+04, 49 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds  

# Display solution values for all variables (with non-zero values):

In [203]:
model.printAttr('x')


    Variable            x 
-------------------------
x[K1,('T1', 'T3')]            1 
x[K1,('T2', 'End')]            1 
x[K1,('T3', 'T2')]            1 
x[K1,('K1', 'T1')]            1 
x[K2,('T1', 'T2')]            1 
x[K2,('T2', 'T3')]            1 
x[K2,('T3', 'End')]            1 
x[K2,('K2', 'T1')]            1 
 x[Start,K1]            1 
 x[Start,K2]            1 
    a[K1,T2]      36.8642 
    a[K1,T3]      76.6734 
    a[K2,T2]      69.2894 
    a[K2,T3]      36.8642 
    d[K1,T1]            1 
    d[K1,T2]            1 
    d[K1,T3]            1 
    d[K2,T1]            1 
    d[K2,T2]            1 
    d[K2,T3]            1 
    d[K1,K1]       150001 
    d[K2,K2]       150001 


In [204]:
model.write("x.lp")